# Analysis of StackOverflow Survey. Part IV 

In this notebook we build a predictiv model for job satisfaction. 

## State the question
I am addressing the third question in this notebook. What can we tell about the job satisfaction of a data coder? What factors do influence it? Also, predict the job satisfaction for a developer who works with big data. 

This is a classification question, we are predicting a satisfaction level for a data developer, which includes: data scientist or machine learning specialist, data or business analyst and data engineer.

## Performance metrics - to review at the end

The following performance measures will be used in this project:
1. Cross validation via StratifiedKFold with 10 folds.
2. Confusion matrix, in particular precision, recall and F1 score.
3. The ROC curve and the related AUC score.

## Gather and prepare the data

Upload the data and keep the subset that contains those developers that work in data science related fields. 

Manually inspect (using the profiling html file, for example) the remaining columns and drop the following columns: 
 - DevClass, Country both introduce noise in the model,
 - JobFactors, Stuck, PlatformWorkedWith they consist of numerous string combinations, as a result of multiple options and choices, the columns have high cardinality and many missing values.

In [ ]:
# create a path string
mypath = os.getcwd()

# upload the datafiles as pandas dataframes
df1 = pd.read_csv(mypath+'/data/survey20_updated.csv', index_col=[0])

# check the uploaded data
df1.shape

In [ ]:
# the data frame that contains the data developers only
df1 = df1[df1.DevClass == 'data_coder']

In [ ]:
# drop the columns mentioned above
df1.drop(columns=['DevClass', 'Country', 'JobFactors', 'Stuck', 'PlatformWorkedWith'], inplace=True)

# check for success
df1.columns

In [ ]:
# create a list of columns to be used in this analysis
#list_cols = ['MainBranch', 'ConvertedComp', 'EdLevel', 'Employment', 'JobSat', 'EdImpt',
       #'Learn', 'Overtime', 'OpSys', 'OrgSize','UndergradMajor', 'WorkWeekHrs']

    # the dataset that contains only the listed columns
#df1 = df1[list_cols]

## Data profiling

In [ ]:
# run this once to generate a profiling report and save it as html file

#import pandas_profiling
#profile = pandas_profiling.ProfileReport(df, minimal=False)
#profile.to_file(output_file="data_coders_report.html")

## Data preprocessing 

### Drop rows based on the Employment column

In [ ]:
# check possible Employment choices
employment = df1.Employment.value_counts().index.to_list()
employment

In [ ]:
# retain only the employed data davelopers
df1 = df1[df1['Employment'] != 'Not employed, but looking for work']

# check for success
df1.Employment.value_counts()

### Create bins for the WorkWeekHrs column

In [ ]:
# create the labels
cut_labels = ['less-10', '10-20', '20-30', '30-40', '40-50', 'more-50']

# define the bins 
m = df1.WorkWeekHrs.max()
cut_bins = [0, 10, 20, 30, 40, 50, m]

# create a new column which contains the new labels
df1['WorkWeek_Bins'] = pd.cut(df1['WorkWeekHrs'], bins=cut_bins, labels=cut_labels)

# check for success
df1['WorkWeek_Bins'].value_counts()

In [ ]:
# change the type of the newly created column
df1['WorkWeek_Bins'] = df1['WorkWeek_Bins'].astype('object')

In [ ]:
# drop the WorkWeekHrs column
df1.drop(columns = 'WorkWeekHrs', inplace=True);

### Create bins for the ConvertedComp column

In [ ]:
# we could use quantile, however I prefer custom bins here
cut_labels = ['less-10K', '10K-30K', '30K-50K', '50K-100K', '100K-200K', 'more-200K']

# define the bins 
m = df1.ConvertedComp.max()
cut_bins = [0, 10000, 30000, 50000, 100000, 200000, m]

# create a new column which contains the new labels
df1['Comp_Bins'] = pd.cut(df1['ConvertedComp'], bins=cut_bins, labels=cut_labels)

# change the type of the newly created column
df1['Comp_Bins'] = df1['Comp_Bins'].astype('object')

# drop the WorkWeekHrs column
df1.drop(columns = 'ConvertedComp', inplace=True);

### Create bins for the Age column

In [ ]:
# define the bin edges
cut_labels = ['<20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '>80']

# define the bins 
m = df1.Age.max()
cut_bins = [0, 20, 30, 40, 50, 60, 70, 80, m]

# create a new column which contains the new labels
df1['Age_Bins'] = pd.cut(df1['Age'], bins=cut_bins, labels=cut_labels)

# change the type of the newly created column
df1['Age_Bins'] = df1['Age_Bins'].astype('object')

# drop the WorkWeekHrs column
df1.drop(columns = 'Age', inplace=True);

### Remove the rows that contain mostly missing values

In [ ]:
# keep only the rows with at least 10 non-NA values
df1.dropna(thresh=10)

# check the result
df1.shape

In [ ]:
df1.JobSat.isnull().sum()

In [ ]:
df1.dropna( how='any', subset=['JobSat'], inplace=True)

In [ ]:
df1.JobSat.isnull().sum()

### Remove duplicates

In [ ]:
# drop duplicate rows, if any
df1.drop_duplicates(subset=None, keep='first', inplace=True)
df1.shape

## Create features and target

Create a dataframe (X) with the features and a pandas series (y) that contains the labels.

In [ ]:
# create a copy of the pre-processed dataframe
df2 = df1.copy()

In [ ]:
# create the predictors dataframe
X = df2.drop(columns = 'JobSat')

# create the labels
y = df2['JobSat']

# check for success
X.info(), len(y)

In [ ]:
# format all the fields as strings in the feature matrix
X = X.astype(str)

## Sample data

We will use $30 \%$ data for testing:

In [ ]:
# split the data in train and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# summarize the data
print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

## Impute missing values

Now that we have test and train data, we can impute missing values on the training set, and use the trained imputer to fill in the test dataset. I will use the KNN imputer from sklearn.

In [ ]:
# create an instance of the imputer
#imputer = KNNImputer(n_neighbors=5)

# fit the imputer on the dataset
#X_train_trans = pd.DataFrame(imputer.fit_transform(X_train), columns = X_train.columns)

# check for success
#X_train_trans.isna().any()
from sklearn.impute import SimpleImputer
def impute_predictors(X_train, X_test):
    imputer = SimpleImputer(strategy='constant', fill_value='missing')
    imputer.fit(X_train)
    X_train_trans = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)
    X_test_trans = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)
    return X_train_trans, X_test_trans

In [ ]:
X_train_trans, X_test_trans = impute_predictors(X_train, X_test)

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, [0])
    ])

In [ ]:
X_train_trans.isna().sum()

## Encode the data

The best practice when encoding variables is to fit the encoding on the training dataset, then apply it to the train and test datasets.

The function below named prepare_inputs() takes the input data for the train and test

In [ ]:
 # prepare input data
def encode_predictors(X_train, X_test):
	enc = OneHotEncoder(handle_unknown='ignore')
	enc.fit(X_train)
	X_train_enc = enc.transform(X_train)
	X_test_enc = enc.transform(X_test)
	return X_train_enc, X_test_enc

In [ ]:
X_train_enc, X_test_enc = encode_predictors(X_train_trans, X_test_trans)

In [ ]:
X_train_enc

Regarding the output data, the target, since it is already encoded as an integer with values from 0 to 5, no other encoding steps are needed at this point.

Alternative would be:

In [ ]:
# prepare target
#def encode_targets(y_train, y_test):
	#le = LabelEncoder()
	#le.fit(y_train)
	#y_train_enc = le.transform(y_train)
	#y_test_enc = le.transform(y_test)
	#return y_train_enc, y_test_enc

### Perform feature selection



In [ ]:
# feature selection
def select_features(X_train, y_train, X_test):
	fs = SelectKBest(score_func=mutual_info_classif, k=20)
	fs.fit(X_train, y_train)
	X_train_fs = fs.transform(X_train)
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs


In [ ]:
X_train_fs, X_test_fs, fs = select_features(X_train_enc, y_train, X_test_enc)

In [ ]:
# what are scores for the features
#for i in range(len(fs.scores_)):
	#print('Feature %d: %f' % (i, fs.scores_[i]))

In [ ]:
# plot the scores
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)


In [ ]:
mi = pd.Series(fs.scores_)


## Refactor code


In [ ]:
X_train_fs

In [ ]:
# create a path string
mypath = os.getcwd()

# read the data from the file
df = pd.read_csv(mypath+'/data/survey20_updated.csv')
# preprocess, split and process data
preproc_df = uf.preprocess_data(df)
X_train, y_train, X_test, y_test = uf.process_data(preproc_df, 'JobSat')

## Baseline model: K NearestNeighbors

In [ ]:
# create an instance of the classifier
knn_clf = KNeighborsClassifier(n_neighbors=5)

# fit the classifier
knn_clf.fit(X_train_fs, y_train)

# predict output values
y_pred = knn_clf.predict(X_test_fs)

In [ ]:
# print evaluation metrics and results

result1 = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result1)

result2 = classification_report(y_test, y_pred)
print('\nClassification Report:')
print (result2)

result3 = accuracy_score(y_test,y_pred)  
print('Accuracy: %.3f' %result3)
         

## Several other algorithms 

In [ ]:
from sklearn.svm import SVC

# create classifier instance
svm_clf = SVC(gamma="auto", random_state=42)
# fit the model
svm_clf.fit(X_train_fs, y_train)

In [ ]:
# predict on the test set
y_pred = svm_clf.predict(X_test_fs)

# test one value
y_test.iloc[20],  y_pred[20]

In [ ]:
some_digit_scores = svm_clf.decision_function(X_test_fs)
some_digit_scores[20]

In [ ]:
np.argmax(some_digit_scores[20])

In [ ]:
svm_clf.classes_

In [ ]:
y_test.values[0]

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(svm_clf, X_test_fs, y_test, cv=10, scoring="accuracy")

In [ ]:
# print evaluation metrics and results

result1 = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result1)

result2 = classification_report(y_test, y_pred, zero_division=0)
print('\nClassification Report:')
print (result2)

result3 = accuracy_score(y_test,y_pred)  
print('Accuracy: %.3f' %result3)
         

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
ovr_clf = OneVsRestClassifier(SVC(gamma="auto", random_state=42))
ovr_clf.fit(X_train_fs, y_train)
y_pred = ovr_clf.predict(X_test_fs)

In [ ]:
result1 = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result1)

result2 = classification_report(y_test, y_pred, zero_division=0)
print('\nClassification Report:')
print (result2)

result3 = accuracy_score(y_test,y_pred)  
print('Accuracy: %.3f' %result3)

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Always scale the input. The most convenient way is to use a pipeline.
sgd_clf = make_pipeline(StandardScaler(with_mean=False),SGDClassifier(max_iter=1000, tol=1e-3))
sgd_clf.fit(X_train_fs, y_train)
y_pred = sgd_clf.predict(X_test_fs)

In [ ]:
result1 = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(result1)

result2 = classification_report(y_test, y_pred, zero_division=0)
print('\nClassification Report:')
print (result2)

result3 = accuracy_score(y_test,y_pred)  
print('Accuracy: %.3f' %result3)

In [ ]:
X = pd.concat([X_train_fs, X_test_fs])
y = pd.concat([y_train, y_test])

In [ ]:
from sklearn import model_selection
from sklearn.linear_model import (LogisticRegression)
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import (KNeighborsClassifier)
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import (RandomForestClassifier)
import xgboost as xgb
from sklearn.metrics import classification_report

In [ ]:
for model in [DecisionTreeClassifier, KNeighborsClassifier, GaussianNB, SVC, 
              RandomForestClassifier, SGDClassifier]:
    make_pipeline(StandardScaler(),model())
    classifier = model()
    kfold = model_selection.KFold(n_splits=10)
    classifier.fit(X_train_fs.toarray(), y_train)
    s = model_selection.cross_val_score(classifier, X_test_fs.toarray(),y_test, cv=kfold)
    #result2 = classification_report(y_test, y_pred, zero_division=0)
    #s = model_selection.cross_val_score(cls, X, y, cv=kfold)
    print(f"{model.__name__:22}  CV_Mean:" f"{s.mean():.3f} CV_STD: {s.std():.2f}")

In [ ]:
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X_train_fs, y_train)


In [ ]:
y_pred = xgb_model.predict(X_test_fs)

print(confusion_matrix(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
import numpy as np

from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

In [ ]:
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)

params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, 
                            n_iter=200, cv=3, verbose=1, n_jobs=1, return_train_score=True)

search.fit(X_train_fs, y_train)

#report_best_scores(search.cv_results_, 1)

In [ ]:
search.cv_results_['mean_test_score']

In [ ]:
search.best_params_

In [ ]:
#from stack
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic')
param_dist = {'n_estimators': stats.randint(150, 1000),
              'learning_rate': stats.uniform(0.01, 0.59),
              'subsample': stats.uniform(0.3, 0.6),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.4),
              'min_child_weight': [1, 2, 3, 4]
             }

numFolds = 5
kfold_5 = cross_validation.KFold(n = len(X), shuffle = True, n_folds = numFolds)

clf = RandomizedSearchCV(clf_xgb, 
                         param_distributions = param_dist,
                         cv = kfold_5,  
                         n_iter = 5, # you want 5 here not 25 if I understand you correctly 
                         scoring = 'roc_auc', 
                         error_score = 0, 
                         verbose = 3, 
                         n_jobs = -1)

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores = []

for train_index, test_index in kfold.split(X_train_fs):   
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
    xgb_model.fit(X_train_fs, y_train)
    
    y_pred = xgb_model.predict(X_test_fs)
    
    scores.append(mean_squared_error(y_test, y_pred))
    
display_scores(np.sqrt(scores))

In [ ]:
cls = RandomForestClassifier()
kfold = model_selection.KFold(n_splits=10)
s = model_selection.cross_val_score(cls, X,y, cv=kfold)

In [ ]:
cls_rf = RandomForestClassifier()
kfold = model_selection.KFold(n_splits=10)
cls_rf.fit(X_train, y_train)
y_pred = cls_rf.predict(X_test)
s = model_selection.cross_val_score(cls_rf, X_test,y_test, cv=kfold)

In [ ]:
print(classification_report(y_test, y_pred, target_names=labels))

In [ ]:
df1 = pd.read_csv(mypath+'/data/survey20_updated.csv')

In [ ]:
df1.shape

In [ ]:
df1 = df1[df1['JobSat'] !=0]

In [ ]:
df1.shape

In [ ]:
df1.JobSat.value_counts()

In [ ]:
# read the data from the file
#df = pd.read_csv(mypath+'/data/survey20_updated.csv')
# preprocess, split and process data
preproc_df1 = uf.preprocess_data(df1)
X1_train, y1_train, X1_test, y1_test = uf.process_data(preproc_df1, 'JobSat')

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Always scale the input. The most convenient way is to use a pipeline.
sgd_clf = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))
sgd_clf.fit(X1_train, y1_train)
y1_pred = sgd_clf.predict(X1_test)

In [ ]:
result1 = confusion_matrix(y1_test, y1_pred)
print('Confusion Matrix:')
print(result1)

result2 = classification_report(y1_test, y1_pred, zero_division=0)
print('\nClassification Report:')
print (result2)

result3 = accuracy_score(y1_test,y1_pred)  
print('Accuracy: %.3f' %result3)

In [ ]:
# create a path string
mypath = os.getcwd()

df2 = pd.read_csv(mypath+'/data/survey20_updated.csv')
df2.columns

In [ ]:
# preprocess, split and process data
#preproc_df2 = uf.preprocess_data(df2)
X2_train, y2_train, X2_test, y2_test = uf.process_data(preproc_df2, 'JobSat')

In [ ]:
X2 = pd.concat([X2_train, X2_test])
y2 = pd.concat([y2_train, y2_test])

In [ ]:
X2.shape

In [ ]:
for model in [DecisionTreeClassifier, KNeighborsClassifier, GaussianNB, SVC, 
              RandomForestClassifier, SGDClassifier]:
    make_pipeline(StandardScaler(),model())
    classifier = model()
    kfold = model_selection.KFold(n_splits=10)
    classifier.fit(X2, y2)
    s = model_selection.cross_val_score(classifier, X2,y2, cv=kfold)
    #result2 = classification_report(y_test, y_pred, zero_division=0)
    #s = model_selection.cross_val_score(cls, X, y, cv=kfold)
    print(f"{model.__name__:22}  CV_Mean:" f"{s.mean():.3f} CV_STD: {s.std():.2f}")

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Always scale the input. The most convenient way is to use a pipeline.
sgd_clf = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))
sgd_clf.fit(X2_train, y2_train)
y2_pred = sgd_clf.predict(X2_test)

In [ ]:
result1 = confusion_matrix(y2_test, y2_pred)
print('Confusion Matrix:')
print(result1)

result2 = classification_report(y2_test, y2_pred, zero_division=0)
print('\nClassification Report:')
print (result2)

result3 = accuracy_score(y2_test,y2_pred)  
print('Accuracy: %.3f' %result3)